# Pré-processamento e Modelagem - Detecção de Fraudes em Cartões de Crédito

Este notebook tem como objetivo realizar o pré-processamento dos dados e a construção de modelos de Machine Learning para detecção de fraudes em transações de cartão de crédito.

## Objetivos desta etapa:

- Separar variáveis preditoras (features) e variável alvo (target).
- Dividir o conjunto de dados em treino e teste.
- Aplicar o escalonamento das variáveis numéricas.
- Tratar o desbalanceamento das classes (fraudes x normais).
- Treinar e avaliar modelos como Random Forest e XGBoost.
- Escolher o modelo de melhor performance para deploy.

## Observação:

Neste projeto, a atenção será voltada para métricas apropriadas a problemas de detecção de anomalias, como:
- Recall
- Precision
- F1-Score
- AUC-ROC

---



In [1]:
import pandas as pd
df = pd.read_csv('../data/creditcard.csv', sep =',')

In [2]:
# Features
X = df.drop('Class', axis=1)

# Variável alvo
y = df['Class']

In [3]:
from sklearn.model_selection import train_test_split

# 70% treino, 30% teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y 
)

In [4]:
from sklearn.preprocessing import StandardScaler

# Inicializar o scaler
scaler = StandardScaler()

# Ajustar o scaler apenas nos dados de treino e transformar
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de teste com o mesmo scaler
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# 1. Treinar o modelo
modelo = RandomForestClassifier(random_state=42, class_weight='balanced')
modelo.fit(X_train_scaled, y_train)


In [ ]:
y_pred = modelo.predict(X_test_scaled)
y_pred_prob = modelo.predict_proba(X_test_scaled)[:,1]
print(confusion_matrix(y_test, y_pred))         
print(classification_report(y_test, y_pred)) 

[[85292     3]
 [   43   105]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.97      0.71      0.82       148

    accuracy                           1.00     85443
   macro avg       0.99      0.85      0.91     85443
weighted avg       1.00      1.00      1.00     85443



In [8]:
auc_score = roc_auc_score(y_test, y_pred_prob)
print(f"AUC-ROC Score: {auc_score:.4f}")

AUC-ROC Score: 0.9275


In [9]:
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.97      0.71      0.82       148

    accuracy                           1.00     85443
   macro avg       0.99      0.85      0.91     85443
weighted avg       1.00      1.00      1.00     85443



### XGBOOST 

In [11]:
from xgboost import XGBClassifier
# Calcular o peso de classes (classe 0 / classe 1)
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# Inicializar o modelo
xgb_model = XGBClassifier(
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100
)

# Treinar o modelo
xgb_model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_prob_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

print("\nRelatório de Classificação (XGBoost):")
print(classification_report(y_test, y_pred_xgb))

auc_xgb = roc_auc_score(y_test, y_pred_prob_xgb)
print(f"\nAUC-ROC Score (XGBoost): {auc_xgb:.4f}")

c:\Users\Victor Bellé\Desktop\Projetos Python\detection-fraud-creditcard\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:31:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Relatório de Classificação (XGBoost):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.88      0.79      0.83       148

    accuracy                           1.00     85443
   macro avg       0.94      0.90      0.92     85443
weighted avg       1.00      1.00      1.00     85443


AUC-ROC Score (XGBoost): 0.9705
